# Agent-Based Modeling with Mesa: Tutorial Notebook

This notebook provides a step-by-step guide to creating an agent-based model using the Mesa library. 
As we progress through the tutorial, we'll cover various aspects of creating, running, and analyzing agent-based models.

**Table of Contents:**

1. [Visualization](#Visualization)

## Boltzmann Wealth Model
This code sets up a visualization interface for the MoneyModel using Mesa's JupyterViz class. It defines parameters like the number of agents and model space dimensions. Agents are depicted in blue, and a histogram of wealth distribution is generated. The interface allows users to adjust parameters and observe wealth distribution in real time.

In [ ]:
from mesa.experimental import JupyterViz

In [ ]:
from mesa.experimental import JupyterViz
from starter_model.money_model import MoneyModel
from matplotlib.figure import Figure
import solara
def agent_portrayal(agent):
    return {
        "color": "tab:blue",
        "size": 50,
    }

model_params = {
    "N": {
        "type": "SliderInt",
        "value": 50,
        "label": "Number of agents:",
        "min": 10,
        "max": 100,
        "step": 1,
    },
    "width": 10,
    "height": 10,
}

def make_histogram(model):
    # Note: you must initialize a figure using this method instead of
    # plt.figure(), for thread safety purpose
    fig = Figure()
    ax = fig.subplots()
    wealth_vals = [agent.wealth for agent in model.schedule.agents]
    # Note: you have to use Matplotlib's OOP API instead of plt.hist
    # because plt.hist is not thread-safe.
    ax.hist(wealth_vals, bins=10)
    solara.FigureMatplotlib(fig)

# This is required to render the visualization in the Jupyter notebook
page = JupyterViz(
    MoneyModel,
    model_params,
    measures=["Gini", make_histogram],
    name="Money Model",
    agent_portrayal=agent_portrayal,
)

page

## Boids flocking model

This code sets up a visualization interface for the BoidFlockers model using Mesa's JupyterViz class. It defines parameters like population size and agent vision range with sliders for user interaction. Agents are portrayed in blue with a size of 5. The interface allows users to adjust parameters and observe the model's behavior in real time.

In [ ]:
from mesa.experimental import JupyterViz
from boid_flockers.model import BoidFlockers

def agent_portrayal(agent):
    return {
        "color": "tab:blue",
        "size": 5,
    }


model_params = {
    "population": {
        "type": "SliderInt",
        "value": 100,
        "label": "Number of agents:",
        "min": 10,
        "max": 200,
        "step": 1,
    },
    "width": 100,
    "height": 100,
    "speed": 2,
    "vision": {
        "type": "SliderInt",
        "value": 10,
        "label": "Vision:",
        "min": 1,
        "max": 15,
        "step": 1,
    },
    "separation": 5,
    'cohere':0.03,
    'separate': 0.02,
    'match':0.05
}

page = JupyterViz(
    model_class=BoidFlockers,
    model_params=model_params,
    measures=[],
    name="BoidFlockers",
    agent_portrayal=agent_portrayal,
)

page

## Predator-prey
The code should set up a Jupyter-based visualization interface for the Wolf-Sheep Predation model. It should import necessary modules, define functions for agent portrayal and visualization, and configure model parameters. Using the `JupyterViz` class from Mesa's experimental module, it should create the visualization page, complete with sliders for parameter adjustment. Additionally, the interface should include a pie chart to visualize the current agent distribution and a line chart to track the population dynamics of wolves and sheep over time.

In [ ]:
from mesa.experimental import JupyterViz
from wolf_sheep.agents import Sheep, Wolf
from wolf_sheep.model import WolfSheep
from matplotlib.figure import Figure
import solara
import mesa

def agent_portrayal(agent):
    if agent is None:
        return
    portrayal = {}
        
    if type(agent) is Sheep:
        portrayal["color"] = "tab:grey"
        portrayal["scale"] = 0.9

    elif type(agent) is Wolf:
        portrayal["color"] = "tab:brown"
        portrayal["scale"] = 0.9

    return portrayal

model_params = {
    "initial_sheep": {
        "type": "SliderInt",
        "value": 100,
        "label": "Initial Sheep Population",
        "min": 100,
        "max": 300,
        "step": 10,
    },
    "sheep_reproduce": {
        "type": "SliderInt",
        "value": 0.04,
        "label": "Sheep Reproduction Rate",
        "min": 0.01,
        "max": 1.0,
        "step": 0.01,
    },
    "initial_wolves": {
        "type": "SliderInt",
        "value": 50,
        "label": "Initial Wolf Population",
        "min": 10,
        "max": 300,
        "step": 1,
    },
    "wolf_reproduce": {
        "type": "SliderInt",
        "value": 0.05,
        "label": "Wolf Reproduction Rate",
        "min": 0.01,
        "max": 1.0,
        "step": 0.01,
    },
    "wolf_gain_from_food": {
        "type": "SliderInt",
        "value": 20,
        "label": "Wolf Gain From Food Rate",
        "min": 1,
        "max": 50,
        "step": 1,
    },
}
